In [1]:
%load_ext autoreload
%autoreload 2

In [66]:
from wdtools import *

In [3]:
# this will take about 9 to 12 minutes
start = time.time()
all_taxlot = combine_taxlot()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 9.0 minutes to complete


In [4]:
setID = 'Set001'

In [81]:
wd = combine_wd_tables(setID=setID, nm_to_add=0)

In [7]:
# this might take a while (> 1 minute)
start = time.time()
df = combined_reindexed_data(setID=setID, nm_to_add=0)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 1.0 minutes to complete


In [8]:
# this might take a while (7 minutes)
start = time.time()
gdf = match_wd_data_with_taxlot(df=df, setID=setID, all_taxlot=all_taxlot, export=True)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 7.0 minutes to complete


In [9]:
unmatched_df = report_unmatched(gdf=gdf, setID=setID, nm_to_add=0, mute = False)

it is about 6.13% of data in the original 1420 records unmatched
there are 1 records (0.07% of the original records) without parcel id


In [10]:
unmatched_df.to_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setID}.csv'), index=False) 

In [11]:
r1_df, r2_df = split_unmatched_df(unmatched_df, ml='N', setID=setID)

In [12]:
# this might take a while (7 minutes)
start = time.time()
rev_r2 = review_unmatched_df_r2(df=r2_df, taxlot=all_taxlot, setID=setID, ml='N', export=True)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

reviewing WD2021-0066
all lots are matched...
corrected trsqq...
reviewing WD2018-0057
all lots are matched...
corrected trsqq...
reviewing WD2022-0172
all lots are matched...
corrected trsqq...
reviewing WD2021-0422
all lots are matched...
corrected trsqq...
reviewing WD2021-0343
all lots are matched...
corrected trsqq...
reviewing WD2017-0313
trsqq matched, checking county code...
county code is corrected, need to check lot numbers...
lot numbers might be wrong...
reviewing WD2021-0551
all lots are matched...
corrected trsqq...
reviewing WD2021-0180
coordinate might be incorrect, nearby taxlot is 1537.00S01.00W21AB--000000806, about 25 ft away
reviewing WD2020-0653
all lots are matched...
corrected trsqq...
reviewing WD2022-0224
all lots are matched...
corrected trsqq...
reviewing WD2017-0347
there is not any matched lot, need to review trsqq, the close-match is 24S32E21
lots to check: ['4900'], and lots to compare: ['100', '200', '300', '400', '500', '600', '700', '800']
reviewing W

In [17]:
wdID = 'WD2021-0422'
url = rev_r2.loc[rev_r2.wetdet_delin_number == wdID, 'DecisionLink'].values[0]
if str(url) == 'nan':
    print('Decision link is not available')
else:
    webbrowser.open(url)

Decision link is not available


In [18]:
rev_r2.loc[rev_r2.wetdet_delin_number == wdID, ['correct_type', 'correction', 'cor_trsqq']]

,correct_type,correction,cor_trsqq
1144,QQ,from 00 to D0,39S20E04D0


In [19]:
def check_review_notes_r2n(wdID, df):
    url = df.loc[df.wetdet_delin_number == wdID, 'DecisionLink'].values[0]
    if str(url) == 'nan':
        print('Decision link is not available')
    else:
        webbrowser.open(url)
    return df.loc[df.wetdet_delin_number == wdID, ['correct_type', 'correction', 'cor_trsqq']]

In [20]:
check_review_notes_r2n(wdID = wdID, df = rev_r2)

Decision link is not available


,correct_type,correction,cor_trsqq
1144,QQ,from 00 to D0,39S20E04D0


In [21]:
def review_loop(df):
    df = df.reset_index()
    for i in range(df.shape[0]):
        wdID = df.loc[i, 'wetdet_delin_number']
        print(wdID)
        print(check_review_notes_r2n(wdID = wdID, df = df))
        user_input = input("Press 'p' to pause or any other key to continue...")
        if user_input == 'p':
            while True:
                user_input = input("Press 'c' to continue...")
                if user_input == 'c':
                    break
        time.sleep(1) # wait for 1 second between iterations

In [22]:
review_loop(df=rev_r2)

WD2021-0066
      correct_type   correction   cor_trsqq
0  range direction  from W to E  18S12E25B0
Press 'p' to pause or any other key to continue...c
WD2018-0057
  correct_type     correction   cor_trsqq
1           QQ  from 00 to BA  07S03W35BA
Press 'p' to pause or any other key to continue...c
WD2022-0172
            correct_type                   correction   cor_trsqq
2  section number and QQ  from 36 and A0 to 35 and B0  36S06W35B0
Press 'p' to pause or any other key to continue...c
WD2021-0422
Decision link is not available
  correct_type     correction   cor_trsqq
3           QQ  from 00 to D0  39S20E04D0
Press 'p' to pause or any other key to continue...C
WD2021-0343
  correct_type     correction   cor_trsqq
4           QQ  from A0 to AA  18S10E10AA
Press 'p' to pause or any other key to continue...c
WD2017-0313
  correct_type                                         correction cor_trsqq
5   lot number  lot number ['3502', '3800'] might be incorrect...  08N07W20
Press 'p' to 

In [38]:
def check_unmatched_r1(wdID, df):
    url = df.loc[df.wetdet_delin_number == wdID, 'DecisionLink'].values[0]
    selcols = ['county', 'trsqq', 'parcel_id', 'latitude', 'longitude', 'record_ID', 'notes']
    if str(url) == 'nan':
        print('Decision link is not available')
    else:
        webbrowser.open(url)
    return df.loc[df.wetdet_delin_number == wdID, selcols]

In [36]:
def review_loop_r1(df):
    for wdID in df.wetdet_delin_number.unique():
        print(wdID)
        print(check_unmatched_r1(wdID = wdID, df = df))
        user_input = input("Press 'p' to pause or any other key to continue...")
        if user_input == 'p':
            while True:
                user_input = input("Press 'c' to continue...")
                if user_input == 'c':
                    break
        time.sleep(1) # wait for 1 second between iterations

In [33]:
r1_df.wetdet_delin_number.unique()

array(['WD2020-0067', 'WD2018-0217', 'WD2018-0306', 'WD2019-0338'],
      dtype=object)

In [39]:
review_loop_r1(df=r1_df)

WD2020-0067
        county      trsqq            parcel_id   latitude   longitude  \
415  Deschutes  17S12E10C  400,500 all partial  44.109403 -121.245841   
416  Deschutes  17S12E10C  400,500 all partial  44.109403 -121.245841   
417  Deschutes  17S12E10C  400,500 all partial  44.109403 -121.245841   

     record_ID    notes  
415        416  Partial  
416        417  Partial  
417        418  Partial  
Press 'p' to pause or any other key to continue...c
WD2018-0217
    county     trsqq parcel_id   latitude   longitude  record_ID notes
204   Coos  24S13W06  101, 100  43.437761 -124.240703        205      
205   Coos  24S13W07       101  43.437546 -124.261227        206      
Press 'p' to pause or any other key to continue...c
WD2018-0306
     county     trsqq parcel_id  latitude  longitude  record_ID    notes
574  Harney  21S37E00      500p   43.8666  -118.2428        575  Partial
576  Harney  20S37E00      100p   43.8666  -118.2428        577  Partial
Press 'p' to pause or any other

In [40]:
setID

'Set001'

In [43]:
cor_r2 = correct_unmatched(r2_df, setID, s='r2', ml='N', export=True)

In [67]:
cor_r2_2 = update_unmatched_df_r2(r2_df, setID, ml='N', export=True)

In [68]:
rev_df = reindex_data(cor_r2_2)

In [69]:
# make sure the matched data before update is correct
matched = match_wd_data_with_taxlot(rev_df, setID, all_taxlot, export=True, update=True)

In [70]:
matched.shape

(2782, 9)

In [71]:
unmatched_df = report_unmatched(gdf=matched, setID=setID, nm_to_add=0, mute = False)

it is about 4.86% of data in the original 1420 records unmatched
there are 1 records (0.07% of the original records) without parcel id


In [72]:
missedID = [rID for rID in cor_r2_2.record_ID.values if rID not in matched.record_ID.values]

In [74]:
len(missedID)

18

In [75]:
outpath = r'L:\NaturalResources\Wetlands\Local Wetland Inventory\WAPO\EPA_2022_Tasks\Task 1 WD Mapping\output'
matched = gpd.read_file(outpath + '\\matched\\matched_records_Set003.shp')

In [78]:
matched.notes.unique()

array(['Partial', None, 'ROW, Partial', 'ROW', 'Partial, Rail',
       'Partial, Water', 'Water', 'ROW, Partial, Rail',
       'ROW, Partial, Many', nan], dtype=object)

In [76]:
matched_toReview = matched[matched.notes.notnull()] 

In [79]:
matched_toReview.notes.unique()

array(['Partial', 'ROW, Partial', 'ROW', 'Partial, Rail',
       'Partial, Water', 'Water', 'ROW, Partial, Rail',
       'ROW, Partial, Many'], dtype=object)

In [77]:
len(matched_toReview.wdID.unique())

263

In [82]:
wd_toReview = wd[wd.wetdet_delin_number.isin(matched_toReview.wdID.unique())]

In [83]:
wd_toReview.to_csv(outpath + f'\\to_review\\partial_matched_{setID}.csv', index=False)

In [84]:
unmatched_df.shape

(69, 25)

In [85]:
unmatched_df.to_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setID}_2.csv'), index=False)

In [86]:
r1_t, r2_t = split_unmatched_df(unmatched_df, ml='Y', setID=setID)

In [87]:
rev_r2_y = review_unmatched_df_r2(df=r2_t, taxlot=all_taxlot, setID=setID, ml='Y', export=True)

reviewing WD2019-0124
there is not any matched lot, need to review trsqq, the close-match is 36S02W21
lots to check: ['ROADS'], and lots to compare: ['100', '101', '102', '199', '200', '300', '400', '500', '600', '700', '800', '900', '1000', '1001', '1002', '1100', '1200', '1201', '1202', '1203', '1204', '1300', '1301', '1302', '1303', '1400', '1500', '1600', '1700']
reviewing WD2021-0703
there is not any matched lot, need to review trsqq, the close-match is 39S20E
lots to check: [], and lots to compare: ['100', '200', '300', '400', '500', '600', '700', '800', '900', '1000', '1001', '1002', '1003', '1004', '1200', '1300', '1600', '1700', '1900', '2300', '2400', '2500', '2600', '2601', '2603', '2604', '2801', '2802', '2900', '3100', '3200', '3300', '3301', '3302', '3303', '3400', '3500', '3600', '3700', '3800', '3900', '4000', '4100', '4200', '4301', '4400', '4500', '4600', '4601', '4700', '4900', '5100', '5200', '5300', '5400', '5500', '5600', '5700', '5800', '5900', '6000', '6100', '6

In [88]:
testdf = rev_r2_y.copy()

In [89]:
taxlot_to_check = get_taxlot_to_check_r2(revdf=testdf, taxlot=all_taxlot, setID=setID, ml='Y')

In [90]:
matchedTxlotIDs = [txlot for txlot in rev_r2_y.ORTaxlot.values if txlot in matched.ORTaxlot.values]

In [91]:
rev_r2_y.loc[rev_r2_y.ORTaxlot.isin(matchedTxlotIDs), 'record_ID'].values

array([ 807, 1148,  209])

In [93]:
res = check_corrected_data(df=df1, setID=setID, all_taxlot=all_taxlot, nm_to_add=0, export=False)

TypeError: match_wd_data_with_taxlot() got an unexpected keyword argument 'nm_to_add'